# Исследование надёжности заёмщиков

## Шаг 1. Знакомство с данными <a class="anchor" id="first"></a>

Импортируем библиотеки и прочитаем файл.

In [43]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
m = Mystem()

df = pd.read_csv('/datasets/data.csv')

Просмотрим первые 10 строк полученной таблицы.

In [44]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Ознакомимся с общей информацией о столбцах.

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Всего в таблице 12 столбцов. Теперь познакомимся с их значениями поближе.

Посмотрим, какие значения содержатся в *children*.


In [46]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Посмотрим, какие значения содержатся в *days_employed*.

In [47]:
print('Значения days_employed между', df['days_employed'].min(),'и', df['days_employed'].max())

Значения days_employed между -18388.949900568383 и 401755.40047533


Переведем полученные значения из дней в годы.

In [48]:
print('Значения days_employed между', df['days_employed'].min() / 365,'и', df['days_employed'].max() / 365)
df['days_employed'].sort_values(ascending = False).head(5) / 365

Значения days_employed между -50.38068465909146 и 1100.6997273296713


6954     1100.699727
10006    1100.591265
7664     1100.479708
2156     1100.477991
7794     1100.448904
Name: days_employed, dtype: float64

Посмотрим, какие значения содержатся в *dob_years*.

In [49]:
print('Значения dob_years между', df['dob_years'].min(),'и', df['dob_years'].max())
print('Второй минимум:', df[df['dob_years']!=0]['dob_years'].min())

Значения dob_years между 0 и 75
Второй минимум: 19


Посмотрим, какие значения содержатся в *education*.

In [50]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Посмотрим, какие значения содержатся в *family_status*.

In [51]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Посмотрим, какие значения содержатся в *gender*.

In [52]:
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

Посмотрим, какие значения содержатся в *income_type*.

In [53]:
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

Посмотрим, какие значения содержатся в *debt*.

In [54]:
df['debt'].unique()

array([0, 1])

Посмотрим, какие значения содержатся в *total_income*.

In [55]:
print('Значения total_income между', df['total_income'].min(),'и', df['total_income'].max())

Значения total_income между 20667.26379327158 и 2265604.028722744


Посмотрим, какие значения содержатся в *purpose*.

In [56]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Вывод**

1. Каждая строка таблицы содержит информацию о клиентах банка - их социальном статусе и других личных характеристиках, а также информацию по платежеспособности. 
2. Наиболее ценные для ответов на поставленные вопросы столбцы в выборке - *debt*, *children*, *family_status*, *total_income*, *purpose*. Однако данные из столбцов *days_employed*, *dob_years*, *education*, *gender*, *income_type* нам также понадобятся для заполнения пропусков и категоризации данных. 
3. Всего 21525 строк, количество значений в столбцах *days_employed* и *total_income* отличается, значит, в них есть пропуски. Также видим, что в *gender* есть значения 'XNA', а в *dob_years* - 0, что также является пропуском.
4. Значения в столбце *days_employed* лучше перевести из дней в года, а в *total_income* сделать целочисленными. 
5. В *children*, *days_employed* содержатся некорректные данные - количество дней трудового стажа и детей не может быть отрицательным, значит, эти значения нужно будет дополнительно обработать. 
6. В *days_employed* максимальное значение рабочего стажа - 401755.40047533 дней или примерно 1100 лет, а в *children* содержится значение 20, что также указывает на некорректные данные, которые нужно будет пересмотреть.
7. Из столбцов *education* и *education_id*, *family_status* и *family_status_id* лучше создать справочники, удалив дубликаты в *education*, созданные заполнением данных в разном регистре. 
8. Поле *purpose* - текстовое поле со свободным вводом, значит, его в целях исследования стоит подвергнуть лемматизации. 

## Шаг 2. Предобработка данных

### Обработка пропусков

Как было выявлено ранее, пропущенные значения встречаются в столбцах *days_employed*, *total_income*, *dob_years*  и *gender*. Три первых столбца содержат количественные значения, в то время как в *gender* данные категориальные. Сперва разберемся с *gender*.

In [58]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Как видно, строк с пропущенным значением в *gender* всего одна. Выведем эту строку.

In [59]:
df[df['gender']=='XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Очевидно, этот пропуск полностью случаен, т.к. ни от чего не зависит. Найдем строки в таблице с похожими на данную параметрами и посмотрим, какой пол встречается чаще всего в такой комбинации. 

In [60]:
df[(df['children']==0) & (df['dob_years']==24) & (df['education_id']==2) & (df['family_status_id']==1)  & (df['income_type']=='компаньон')]



,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
5669,0,-2175.483527,24,неоконченное высшее,2,гражданский брак,1,F,компаньон,0,119537.358630,сыграть свадьбу
5998,0,-219.732426,24,неоконченное высшее,2,гражданский брак,1,F,компаньон,0,250701.133144,строительство жилой недвижимости
10701,0,-2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Абсолютное большинство клиентов с таким набором параметров - женщины, значит, мы можем заменить пропуск на 'F', при этом не потеряв запись и не испортив статистику.

In [61]:
df['gender'] = df['gender'].replace('XNA','F')
df['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

Теперь в *gender* больше нет пропусков. Идем дальше и изучим пропуски в *days_employed* и *total_income*.

In [62]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [63]:
len(df[df['days_employed'].isna() & df['total_income'].isna()])

2174

Пропусков в *days_employed* и *total_income* равное количество, причем они содержатся в одних и тех же строках. Такой пропуск можно объяснить тем, что у некоторых клиентов нет дохода и рабочего стажа, т.к. они никогда не работали, и тогда NaN можно заменить на 0. Проверим эту гипотезу, просмотрев для этих строк значения в столбцах *income_type*.

In [64]:
df[df['days_employed'].isna() & df['total_income'].isna()]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Наша гипотеза неверна, т.к. у данных клиентов указаны типы занятости, предполгающие наличие стажа и дохода. Значит, пропуски в данных строках обусловлены иными причинами - напимер, техническими, или же статистика собирается только по клиентам, предоставившим документы, подтверждающие стаж и доход. В любом случае пропущенные данные придется заполнить характерными значениями.

Найдем средний трудовой стаж по типу занятости, но сначала переведем значения в *days_employed* из дней в года и запишем их в новый столбец *years_employed*.

In [65]:
df['days_employed'] = df['days_employed']/365
df.rename(columns={'days_employed': 'years_employed'}, inplace=True)

Теперь переведем непустые значения *years_employed* в тип данных int, чтобы получить количество полных лет стажа.

In [66]:
df['years_employed'] = df['years_employed'].dropna().astype('int')
df.groupby('income_type')['years_employed'].mean()

income_type
безработный        1003.500000
в декрете            -9.000000
госслужащий          -8.813262
компаньон            -5.277911
пенсионер           999.513506
предприниматель      -1.000000
сотрудник            -5.875974
студент              -1.000000
Name: years_employed, dtype: float64

Видим, что в данных ошибка. По безработным и пенсионерам цифры слишком большие, и нам остается лишь предполагать, что послужило причиной такой ошибки. Ошибки и в остальных типах занятости - вместо положительных значений в таблицу занесены отрицательные. Такие разные ошибки в одном столбце могли появиться, если данные по трудовому стажу брали из разных источников - отдельно для пенсионеров и безработных и отдельно для других категорий. Ошибка с отрицательными значениями могла получиться, если при нахождении трудового стажа перепутали местами дату начала работы и дату окончания работы. Исправим эту ошибку, а значения для пенсионеров и безработных оставим пока пустыми. 

In [67]:
def fix_years_employed(years_employed):
    if years_employed < 0:
        return years_employed*(-1)
    elif years_employed == 0:
        return years_employed
    else:
        return np.nan
    
df['years_employed'] = df['years_employed'].apply(fix_years_employed)
df.groupby('income_type')['years_employed'].mean()

income_type
безработный             NaN
в декрете          9.000000
госслужащий        8.813262
компаньон          5.277911
пенсионер               NaN
предприниматель    1.000000
сотрудник          5.875974
студент            1.000000
Name: years_employed, dtype: float64

Заполним отсутствующие значения в *years_employed*. Очевидно, что стаж - это разность текущего возраста и возраста, когда человек начал работать, поэтому возраст играет ключевую роль во влиянии на количество лет стажа, однако момент старта работы может меняться также в зависимости от типа занятости. Поэтому характерные значения для стажа будем искать, разбив данные на категории по значениям этих характеристик. Но не забудем и о том, что в *dob_years* содержатся пропуски, и предварительно отфильтруем нули. Для каждой категории найдем медианный стаж и результаты запишем в сводную таблицу. В данном случае корректнее применять медиану, т.к. распределение стажа клиентов не является нормальным.

In [68]:
years_employed_pivot = df[df['dob_years']!=0].pivot_table(index = 'dob_years', columns = 'income_type',
values = 'years_employed', aggfunc = 'median')
years_employed_pivot = years_employed_pivot.reset_index()
years_employed_pivot.head()

income_type,dob_years,в декрете,госслужащий,компаньон,предприниматель,сотрудник,студент
0,19,NaN,1.0,1.0,NaN,2.0,NaN
1,20,NaN,1.0,1.0,NaN,2.0,NaN
2,21,NaN,3.0,1.0,NaN,1.0,NaN
3,22,NaN,1.0,2.0,NaN,1.0,1.0
4,23,NaN,1.0,1.0,NaN,2.0,NaN


Мы получили сводную таблицу, но также необходимо предусмотреть случаи, когда для определенного типа занятости не найдется клиентов с нужным возрастом (как, например, для клиентов в декрете и предпринимателей - их не так много), а также случаи с безработными и пенсионерами, ни по одному из которых стаж не известен. Для таких случаев будет достаточно подставить медианное значение стажа по группе клиентов с одинаковым возрастом. Запишем эти значения в отдельный датафрейм. 

In [69]:
df_median_age = df[df['dob_years']!=0].groupby('dob_years')['years_employed'].median()
df_median_age = df_median_age.reset_index()
df_median_age.head()

,dob_years,years_employed
0,19,1.0
1,20,1.0
2,21,1.0
3,22,1.0
4,23,1.0


Напишем и применим к каждой строке датафрейма функцию, которая будет обрабатывать пустые значения стажа, заменяя их на характерные либо из сводной таблицы, либо из датафрейма с медианами. Исключим из обработки клиентов, по которым неизвестен возраст - их мы обработаем позже. Дробные значения будем округлять до целого с помощью *round*.

In [70]:
def fill_years_employed(row):
    dob_years = row['dob_years']
    income_type = row['income_type']
    years_employed = row['years_employed']
    if pd.isna(years_employed) and dob_years!=0:
        if income_type!='пенсионер' or income_type!='безработный' or pd.isna(float(years_employed_pivot[years_employed_pivot['dob_years']==dob_years][income_type])):
            return int(round(df_median_age[df_median_age['dob_years']==dob_years]['years_employed']))
        return int(round(years_employed_pivot[years_employed_pivot['dob_years']==dob_years][income_type]))
    return years_employed
        
df['years_employed'] = df.apply(fill_years_employed, axis=1)

Проверим результат работы функции - найдем все пустые значения в *years_employed*.

In [71]:
df['years_employed'].isna().sum()

27

In [72]:
df[df['dob_years']!=0]['years_employed'].isna().sum()

0

Всего пустых значений осталось 27 - это те самые случаи, когда по клиенту неизвестны ни стаж, ни возраст.

Заполним нулевые значения в *dob_years*. Проанализируем наличие нулей в этом столбце.

In [73]:
df[df['dob_years']==0].groupby('income_type')['dob_years'].count()

income_type
госслужащий     6
компаньон      20
пенсионер      20
сотрудник      55
Name: dob_years, dtype: int64

Нулевые значения появляются рандомно и можно предположить, что это также техническая ошибка или же ошибка ввода.

Как уже описывалось ранее, возраст тесно связан со стажем, который в свою очередь зависит от типа занятости. Поэтому характерные значения для возраста будем искать по аналогичной схеме, разбив данные на категории по значениям этих характеристик для клиентов, у которых известен стаж. Для каждой категории найдем медианный возраст и результаты запишем в сводную таблицу. В данном случае корректнее применять медиану, т.к. распределение возраста клиентов не является нормальным.

In [74]:
dob_years_pivot1 = df[df['years_employed'].isna()==False].pivot_table(index = 'years_employed', columns = 'income_type',
values = 'dob_years', aggfunc = 'median')
dob_years_pivot1 = dob_years_pivot1.reset_index()
dob_years_pivot1.head()

income_type,years_employed,безработный,в декрете,госслужащий,компаньон,пенсионер,предприниматель,сотрудник,студент
0,0.0,NaN,NaN,35.0,35.5,NaN,NaN,36.0,NaN
1,1.0,NaN,NaN,36.5,35.0,22.0,27.0,36.0,22.0
2,2.0,NaN,NaN,34.0,35.0,26.0,NaN,35.0,NaN
3,3.0,31.0,NaN,31.5,34.0,31.5,NaN,33.0,NaN
4,4.0,NaN,NaN,36.0,38.0,71.0,NaN,37.0,NaN


Как и в случае со стажем, нужно учесть наличие клиентов, для которых не найдется характерного значения в сводной таблице. Но здесь мы не сможем обойтись одним стажем, т.к. существуют 27 случаев, когда неизвестны ни стаж, ни возраст. Для таких клиентов нужно подобрать возраст в зависимости от других параметров - например, пола, семейного статуса, количества детей и типа занятости - все эти показатели в совокупности могут косвенно указыввать на возраст клиента. Составим такую сводную таблицу, но для начала разберемся с некорректным количеством детей в *children*.

В *children* есть два значения, которые нас не устраивают - "-1" и "20". Скорее всего, они возникли по вине технических ошибок или ошибок ввода. "20" могло получиться из-за неверной обработки значения "2.0", а отрицательное "-1" получилось из положительного "1", как и в случае со стажем. Исправим ошибки.

In [75]:
def fix_children(children):
    if children == -1:
        return 1
    elif children == 20:
        return 2
    return children
df['children'] = df['children'].apply(fix_children)
df['children'].unique()

array([1, 0, 3, 2, 4, 5])

Теперь в *children* только корректные значения. Перейдем к созданию сводной таблицы с медианным возрастом.

In [76]:
dob_years_pivot2 = df.pivot_table(index = ['gender','family_status_id','children'], columns = 'income_type',
values = 'dob_years', aggfunc = 'mean')
dob_years_pivot2 = dob_years_pivot2.reset_index()
dob_years_pivot2.head()

income_type,gender,family_status_id,children,безработный,в декрете,госслужащий,компаньон,пенсионер,предприниматель,сотрудник,студент
0,F,0,0,NaN,NaN,45.166205,43.375629,58.941457,NaN,43.773576,NaN
1,F,0,1,NaN,NaN,37.000000,36.492813,56.990826,NaN,37.039423,NaN
2,F,0,2,NaN,39.0,34.532609,35.255411,49.916667,NaN,34.306759,NaN
3,F,0,3,NaN,NaN,36.800000,35.225806,50.500000,NaN,34.279070,NaN
4,F,0,4,NaN,NaN,34.000000,NaN,NaN,NaN,35.800000,NaN


Напишем и применим к каждой строке датафрейма функцию, которая будет обрабатывать пустые значения возраста, заменяя их на характерные либо из первой, либо из второй сводной таблицы. Дробные значения будем округлять до целого с помощью *round*.

In [77]:
def fill_dob_years(row):
    dob_years = row['dob_years']
    income_type = row['income_type']
    years_employed = row['years_employed']
    gender = row['gender']
    family_status_id = row['family_status_id']
    children = row['children']
    if dob_years==0:
        if pd.isna(years_employed)==False:
            return int(round(dob_years_pivot1[dob_years_pivot1['years_employed']==years_employed][income_type]))
        else:
            return int(round(dob_years_pivot2[(dob_years_pivot2['gender']==gender)&(dob_years_pivot2['family_status_id']==family_status_id)&(dob_years_pivot2['children']==children)][income_type]))
    return dob_years
        
df['dob_years'] = df.apply(fill_dob_years, axis=1)

Проверим, остались ли нулевые значения в *dob_years*.

In [78]:
df[(df['dob_years']==0)]

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Нулевых значений не осталось.

Применим повторно фнукцию *fill_years_employed* для заполнения пустых значений стажа, чтобы обработать оставшиеся 27 случаев.

In [79]:
df['years_employed'] = df.apply(fill_years_employed, axis=1)
df[(df['years_employed'].isna())]

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Пустых значений в *years_employed* не осталось.

Теперь, когда по всем клиентам есть значения стажа, заполним отсутствующие значения дохода. Считая, что доход зависит от уровня образования, типа занятости и стажа клиента, заполним сводную таблицу медианными значениями.

In [80]:
total_income_pivot = df.pivot_table(index = ['education_id','years_employed'], columns = 'income_type',
values = 'total_income', aggfunc = 'median')
total_income_pivot = total_income_pivot.reset_index()
total_income_pivot.head()

income_type,education_id,years_employed,безработный,в декрете,госслужащий,компаньон,пенсионер,предприниматель,сотрудник,студент
0,0,0.0,NaN,NaN,147746.828911,185492.137323,NaN,NaN,148793.715540,NaN
1,0,1.0,NaN,NaN,168966.283322,192345.942722,NaN,499163.144947,169730.871768,98201.625314
2,0,2.0,NaN,NaN,167779.266433,197112.764013,214963.301941,NaN,154485.555057,NaN
3,0,3.0,NaN,NaN,158920.430941,207060.258413,NaN,NaN,165532.734930,NaN
4,0,4.0,NaN,NaN,162007.725708,199950.204937,189309.221437,NaN,161957.222255,NaN


Как и в случае со стажем создадим датафрейм с медианным доходом в группировке только по типу дохода.

In [81]:
df_median_income = df.groupby('income_type')['total_income'].mean()
df_median_income = df_median_income.reset_index()
df_median_income

,income_type,total_income
0,безработный,131339.751676
1,в декрете,53829.130729
2,госслужащий,170898.309923
3,компаньон,202417.461462
4,пенсионер,137127.465690
5,предприниматель,499163.144947
6,сотрудник,161380.260488
7,студент,98201.625314


Напишем и применим к датафрейму функцию, заполняющую пустые значения дохода.

In [82]:
def fill_total_income(row):
    income_type = row['income_type']
    years_employed = row['years_employed']
    education_id = row['education_id']
    total_income = row['total_income']
    if pd.isna(total_income):
        if pd.isna(float(total_income_pivot[(total_income_pivot['years_employed']==years_employed)&(total_income_pivot['education_id']==education_id)][income_type])):
            return int(df_median_income[df_median_income['income_type']==income_type]['total_income'])
        return int(total_income_pivot[(total_income_pivot['years_employed']==years_employed)&(total_income_pivot['education_id']==education_id)][income_type])
    return total_income
        
df['total_income'] = df.apply(fill_total_income, axis=1)
df[df['total_income'].isna()]


,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Пустых значений в *total_income* не осталось.



**Вывод**

На этапе предобработки в данных обнаружились пропуски, которые были заполнены характерными значениями. Для каждого вида пропусков были разработаны алгоритмы, задействующие зависимые/зависящие столбцы и метод поиска медианного значения. Вот такие зависимости были использованы:

| Данные      | Зависимость (в первую очередь)                                          | Зависимость (во вторую очередь)                           |
| :----------- | :------------------------------------------------------- |:----------------------------------------------------- |
| Пол         | Количество детей, возраст, уровень образования, семейное положение, тип занятости|                                                     |
| Возраст      | Стаж, тип занятости                                        | Пол, количество детей, семейное положение, тип занятости|
| Стаж        | Возраст, тип занятости                                                  | Возраст                                               |
| Доход       | Стаж, уровень образования, тип занятости                                   | Тип занятости                                         |

### Замена типа данных

Теперь, когда пропусков не осталось, заменим значения с плавающей точкой на целочисленные в *years_employed* и *total_income*. Для этого используем метод *astype* с аргументом *int*. Для *years_employed* он просто превратит числа в целые, т.к. мы уже округляли их на этапе заполнения пропусков. Для *total_income* будет произведено округление в меньшую сторону, что не противоречит логике этого поля.

In [84]:
df['years_employed'] = df['years_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.head()

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,15,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,6,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Значения в столбцах *years_employed* и *total_income* были с плавающей точкой, мы заменили их на целочисленные для удобства обработки и экономии памяти.

### Обработка дубликатов

Создадим справочники для столбцов *family_status* и *education*.

In [86]:
family_status_dict = df[['family_status','family_status_id']].drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [87]:
education_dict = df[['education','education_id']].drop_duplicates().reset_index(drop=True)
education_dict

,education,education_id
0,высшее,0
1,среднее,1
2,Среднее,1
3,СРЕДНЕЕ,1
4,ВЫСШЕЕ,0
5,неоконченное высшее,2
6,начальное,3
7,Высшее,0
8,НЕОКОНЧЕННОЕ ВЫСШЕЕ,2
9,Неоконченное высшее,2


Приведем все строки в *family_status* и *education* к нижнему регистру.

In [88]:
family_status_dict['family_status'] = family_status_dict['family_status'].str.lower()
education_dict['education'] = education_dict['education'].str.lower()

В *education_dict* есть дубликаты, вызванные вводом значений в разном регистре. Теперь, когда мы привели все строки к нижнему регистру, избавимся от дубликатов, применив метод *drop_duplicates*.

In [89]:
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


Подставим в датафрейм корректные значения.

In [90]:
df2 = df.merge(family_status_dict, on='family_status_id', how='left')
df['family_status'] = df2['family_status_y']
df2 = df.merge(education_dict, on='education_id', how='left')
df['education'] = df2['education_y']

Проверим, что все получилось.

In [91]:
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [92]:
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

**Вывод**

Строковые значения, обозначающие уровень образования и семейное положение, были введены в разном регитсре, что могло послужить причиной возникновения дублей при дальнейшей группировке данных по этим полям. С помощью методов *lower* и *drop_duplicates* мы предотвратили эту ситуацию.

### Лемматизация

В столбце *purpose* собраны ответы клиентов, обозначающие цель кредита. Выберем уникальные значения этого столбца и выделим основные леммы с помощью метода *lemmatize*.

In [55]:
purpose = df[['purpose']].drop_duplicates().reset_index(drop=True)


def find_purpose_object(text):
    lemmas = m.lemmatize(text)
    if 'жилье' in lemmas or 'недвижимость' in lemmas:
        return 'недвижимость'
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    elif 'образование' in lemmas:
        return 'образование'
        

purpose['purpose_object'] = purpose['purpose'].apply(find_purpose_object)
purpose.head()

,purpose,purpose_object
0,покупка жилья,недвижимость
1,приобретение автомобиля,автомобиль
2,дополнительное образование,образование
3,сыграть свадьбу,свадьба
4,операции с жильем,недвижимость


Категория "недвижимость" - самая разнообразная, её интересно рассмотреть ещё в нескольких разрезах - какую именно недвижимость хочет приобрести клиент и с какой целью. Напишем функции, выделяющие эти категоии.

In [56]:
def find_purpose_property_type(row):
    purpose = row['purpose']
    purpose_object = row['purpose_object']
    lemmas = m.lemmatize(purpose)
    if purpose_object == 'недвижимость':
        if 'жилье' in lemmas:
            if 'сдача' in lemmas:
                return 'инвестиции'
            return 'жилье для себя'
        elif 'коммерческий' in lemmas:
            return 'коммерческая недвижимость'
        return 'без уточнения'
    return np.nan

def find_purpose_property_operation(row):
    purpose = row['purpose']
    purpose_object = row['purpose_object']
    lemmas = m.lemmatize(purpose)
    if purpose_object == 'недвижимость':
        if 'покупка' in lemmas or 'приобретение' in lemmas:
            return 'покупка'
        elif 'строительство' in lemmas:
            return 'строительство'
        elif 'ремонт' in lemmas:
            return 'ремонт'
        return 'без уточнения'
    return np.nan

purpose['purpose_property_type'] = purpose.apply(find_purpose_property_type, axis = 1)
purpose['purpose_property_operation'] = purpose.apply(find_purpose_property_operation, axis = 1)
purpose

,purpose,purpose_object,purpose_property_type,purpose_property_operation
0,покупка жилья,недвижимость,жилье для себя,покупка
1,приобретение автомобиля,автомобиль,NaN,NaN
2,дополнительное образование,образование,NaN,NaN
3,сыграть свадьбу,свадьба,NaN,NaN
4,операции с жильем,недвижимость,жилье для себя,без уточнения
5,образование,образование,NaN,NaN
6,на проведение свадьбы,свадьба,NaN,NaN
7,покупка жилья для семьи,недвижимость,жилье для себя,покупка
8,покупка недвижимости,недвижимость,без уточнения,покупка
9,покупка коммерческой недвижимости,недвижимость,коммерческая недвижимость,покупка


Теперь добавим к основной таблице полученный справочник по целям.

In [57]:
df = df.merge(purpose, on='purpose', how='left')
df.head(10)

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_object,purpose_property_type,purpose_property_operation
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,жилье для себя,покупка
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,NaN,NaN
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,жилье для себя,покупка
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,NaN,NaN
4,0,6,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,NaN,NaN
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,жилье для себя,покупка
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,жилье для себя,без уточнения
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,NaN,NaN
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,NaN,NaN
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,жилье для себя,покупка


**Вывод**

Данные в столбце *purpose* требовали выделения основных целей кредита. Чтобы выделить их, мы лемматизировали данные с помощью метода *lemmatize* библиотеки pymystem3 и в итоге получили всего 4 основных цели - недвижимость, свадьба, образование и автомобиль. Также категорию "Недвижимость" разбили ещё на несколько категорий по типу недвижимости и операциям с ней.

### Категоризация данных

В нашем датафрейме есть данные, которые нужно категоризировать, т.к. они могут присутствовать в наборе только единожды и по ним нельзя будет сделать статистические выводы. Это касается столбцов *children*, *total_income* и *family_status*.

Сделаем две категоризации столбца *children*. Сначала добавим отдельный столбец, где разобьем значения *children* на группы в зависимости от наличия детей, а столбец с количественными значениями категоризируем на более детальные группы - нет детей, 1, 2, 3 и более детей.

In [58]:
def children_grouped1(children):
    if children == 0:
        return 'Нет детей'
    return 'Есть дети'

def children_grouped2(children):
    if children == 0:
        return 'Нет детей'
    elif children == 1:
        return '1 ребенок'
    elif children == 2:
        return '2 ребенка'
    elif children >= 3:
        return '3 и более детей'
    
df['children_grouped'] = df['children'].apply(children_grouped1)

In [59]:
df['children'] = df['children'].apply(children_grouped2)

Разобьем *family_status* на две группы по наличию у клиента партнера.

In [60]:
def family_status_grouped(family_status_id):
    if family_status_id == 0 or family_status_id == 1:
        return 'Есть партнер'
    return 'Нет партнера'
    
df['family_status_grouped'] = df['family_status_id'].apply(family_status_grouped)

df['family_status_grouped'].value_counts()

Есть партнер    16557
Нет партнера     4968
Name: family_status_grouped, dtype: int64

Выделим группы дохода.

Чтобы нам было отчего отталкиваться в выделении групп дохода, найдем 25-й, 50-й и 75-й процентили.

In [61]:
np.percentile(df['total_income'], 25)

107643.0

In [62]:
np.percentile(df['total_income'], 50)

143470.0

In [63]:
np.percentile(df['total_income'], 75)

198060.0

Таким образом, ключевые значения дохода - примерно 110, 140 (медиана) и 200 тысяч рублей. Выделим группы, которые будут обозначать следующие категории:
- значительно меньше медианы
- чуть меньше медианы
- чуть больше медианы
- значительно больше медианы

In [64]:
def total_income_grouped(total_income):
    if total_income < 100000:
        return '< 110 тыс.'
    if 100000 <= total_income < 140000:
        return '110-140 тыс.'
    elif 140000 <= total_income < 200000:
        return '140-200 тыс.'
    elif total_income >= 200000:
        return '> 200 тыс.'
    
df['total_income'] = df['total_income'].apply(total_income_grouped)
df['total_income'].value_counts()

140-200 тыс.    6218
110-140 тыс.    5650
> 200 тыс.      5182
< 110 тыс.      4475
Name: total_income, dtype: int64

**Вывод**

Мы категоризировали данные, которые могли помешать нам сделать выводы - выделили категории семейного статуса, значений количества детей и дохода. Это позволит провести анализ точнее. 

## Шаг 3. Ответы на вопросы

Итак, проверим наши гипотезы и ответим на вопрос, зависит ли возврат кредита в срок от количества детей, семейного положения, дохода и цели.

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [65]:
children_grouped_pivot = df.pivot_table(index = 'children_grouped', columns = 'debt',values = 'income_type', aggfunc = 'count', margins=True)
children_grouped_pivot = children_grouped_pivot.reset_index()
children_grouped_pivot['percent'] = round(children_grouped_pivot[0]/children_grouped_pivot['All']*100,2)
children_grouped_pivot.sort_values(by = 'percent', ascending = False)


debt,children_grouped,0,1,All,percent
1,Нет детей,13086,1063,14149,92.49
2,All,19784,1741,21525,91.91
0,Есть дети,6698,678,7376,90.81


In [66]:
children_pivot = df.pivot_table(index = 'children', columns = 'debt',values = 'income_type', aggfunc = 'count', margins=True)
children_pivot = children_pivot.reset_index()
children_pivot['percent'] = round(children_pivot[0]/children_pivot['All']*100,2)
children_pivot.sort_values(by = 'percent', ascending = False)


debt,children,0,1,All,percent
3,Нет детей,13086,1063,14149,92.49
4,All,19784,1741,21525,91.91
2,3 и более детей,349,31,380,91.84
0,1 ребенок,4420,445,4865,90.85
1,2 ребенка,1929,202,2131,90.52


**Вывод**

Результаты показывают, что клиенты без детей вернут кредит в срок с большей вероятностью, чем клиенты с детьми. Однако если семья многодетная (3 и более детей), шанс возвращения кредита в срок выше, чем у клиентов с 1 ребенком или 2 детьми. Причинами данного явления может быть то, что у семей с детьми финансовая нагрузка выше, поэтому кредит выплачивать сложнее. Но многодетным семьям помогает в выплатах государство, что немного завышает статистику по возврату ими кредита.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [67]:
family_status_grouped_pivot = df.pivot_table(index = 'family_status_grouped', columns = 'debt',values = 'income_type', aggfunc = 'count', margins=True)
family_status_grouped_pivot = family_status_grouped_pivot.reset_index()
family_status_grouped_pivot['percent'] = round(family_status_grouped_pivot[0]/family_status_grouped_pivot['All']*100,2)
family_status_grouped_pivot.sort_values(by = 'percent', ascending = False)


debt,family_status_grouped,0,1,All,percent
0,Есть партнер,15238,1319,16557,92.03
2,All,19784,1741,21525,91.91
1,Нет партнера,4546,422,4968,91.51


In [68]:
family_status_pivot = df.pivot_table(index = ['family_status'], columns = 'debt',values = 'income_type', aggfunc = 'count', margins=True)
family_status_pivot = family_status_pivot.reset_index()
family_status_pivot['percent'] = round(family_status_pivot[0]/family_status_pivot['All']*100,2)
family_status_pivot.sort_values(by = 'percent', ascending = False)


debt,family_status,0,1,All,percent
1,вдовец / вдова,897,63,960,93.44
0,в разводе,1110,85,1195,92.89
3,женат / замужем,11449,931,12380,92.48
5,All,19784,1741,21525,91.91
2,гражданский брак,3789,388,4177,90.71
4,не женат / не замужем,2539,274,2813,90.26


**Вывод**

Согласно полученным цифрам, клиенты, у которых есть партнер (жентые/замужем или в гражданском браке) выплачивают кредит лучше, чем те, кто живет один. Это может объясняться тем, что партнер является дополнительной финансовой поддержкой, что способствует выплате кредита. Но если мы посмотрим на статистику по "одиноким", то увидим, что категории разведенных и вдовцов также выплачивают кредит более успешно чем те, кто никогда не был женат/замужем. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [69]:
total_income_pivot = df.pivot_table(index = 'total_income', columns = 'debt',values = 'income_type', aggfunc = 'count', margins=True)
total_income_pivot = total_income_pivot.reset_index()
total_income_pivot['percent'] = round(total_income_pivot[0]/total_income_pivot['All']*100,2)
total_income_pivot.sort_values(by = 'percent', ascending = False)


debt,total_income,0,1,All,percent
3,> 200 тыс.,4819,363,5182,92.99
2,< 110 тыс.,4120,355,4475,92.07
4,All,19784,1741,21525,91.91
1,140-200 тыс.,5685,533,6218,91.43
0,110-140 тыс.,5160,490,5650,91.33


**Вывод**

Среди клиентов, у которых доход значительно меньше или значительно больше медианы, меньше должников, чем среди тех, у кого доход приближен к среднему значению. 

- Как разные цели кредита влияют на его возврат в срок?

In [70]:
purpose_object_pivot = df.pivot_table(index = 'purpose_object', columns = 'debt',values = 'income_type', aggfunc = 'count', margins=True)
purpose_object_pivot = purpose_object_pivot.reset_index()
purpose_object_pivot['percent'] = round(purpose_object_pivot[0]/purpose_object_pivot['All']*100,2)
purpose_object_pivot.sort_values(by = 'percent', ascending = False)


debt,purpose_object,0,1,All,percent
1,недвижимость,10058,782,10840,92.79
3,свадьба,2162,186,2348,92.08
4,All,19784,1741,21525,91.91
2,образование,3652,370,4022,90.80
0,автомобиль,3912,403,4315,90.66


In [71]:
purpose_property_type_pivot = df.pivot_table(index = 'purpose_property_type', columns = 'debt',values = 'income_type', aggfunc = 'count', margins=True)
purpose_property_type_pivot = purpose_property_type_pivot.reset_index()
purpose_property_type_pivot['percent'] = round(purpose_property_type_pivot[0]/purpose_property_type_pivot['All']*100,2)
purpose_property_type_pivot.sort_values(by = 'percent', ascending = False)


debt,purpose_property_type,0,1,All,percent
1,жилье для себя,3564,256,3820,93.30
4,All,10058,782,10840,92.79
0,без уточнения,4677,375,5052,92.58
3,коммерческая недвижимость,1216,99,1315,92.47
2,инвестиции,601,52,653,92.04


In [72]:
purpose_property_operation_pivot = df.pivot_table(index = 'purpose_property_operation', columns = 'debt',values = 'income_type', aggfunc = 'count', margins=True)
purpose_property_operation_pivot = purpose_property_operation_pivot.reset_index()
purpose_property_operation_pivot['percent'] = round(purpose_property_operation_pivot[0]/purpose_property_operation_pivot['All']*100,2)
purpose_property_operation_pivot.sort_values(by = 'percent', ascending = False)


debt,purpose_property_operation,0,1,All,percent
2,ремонт,577,35,612,94.28
1,покупка,4146,310,4456,93.04
4,All,10058,782,10840,92.79
0,без уточнения,3598,293,3891,92.47
3,строительство,1737,144,1881,92.34


**Вывод**

Недвижимость и свадьба - вот цели, кредит на которые выплачивается более успешно, чем тот, что взяли на образование или автомобиль. Это может объясняться тем, что недвижимость - самая серьезная цель, деньги на которую откладываются планомеренно, а кредит на свадьбу чаще берется с расчетом на то, что он окупится после свадьбы, поэтому и погашается вовремя.

А что касается недвижимости, клиенты успешнее выплачивают кредиты, взятые на покупку или ремонт собственного жилья. 

## Шаг 4. Общий вывод

1. Между наличием детей и выплатой кредита в срок есть зависимость - лучше выплачивают кредит клиенты без детей.
2. Клиенты, никогда не бывшие женаты/замужем, выплачивают кредит в срок реже, чем те, кто сейчас состоит в браке или когда-либо состоял в прошлом.
3. Уровень длохода клиентов также влияет на возврат кредита в срок. Клиенты, у которых доход значительно выше или значительно ниже медианного, скорее выплатят кредит в срок, чем те, у кого доход ближе к медиане.
4. Что касается целей, клиенты лучше выплачивают кредиты на покупку или ремонт собственной недвижимости, а также кредиты на свадьбы.